## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
gmaps.__version__

'0.9.0'

In [3]:
from jupyter_core.paths import jupyter_data_dir
jupyter_data_dir()

'/Users/cartoon_air/Library/Jupyter'

In [4]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.

vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,6,Butaritari,KI,82.35,broken clouds,3.0707,172.7902,Isles Sunset Lodge
1,9,Rikitea,PF,76.15,overcast clouds,-23.1203,-134.9692,People ThankYou
2,11,Bengkulu,ID,84.65,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
3,13,Mahebourg,MU,82.69,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
4,15,Olinda,BR,82.13,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda


In [5]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]


In [6]:
# 4a. Add a marker layer for each city to the map.

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

# Call the figure to plot the data.
# 4b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
#df.loc[df['column_name'] == some_value]

vacation_start = vacation_df.loc[vacation_df['City'] == 'Maningrida']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Maningrida']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Alice Springs']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Hervey Bay']
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Mareeba']

vacation_start


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
131,562,Maningrida,AU,88.95,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge


In [8]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start = vacation_start.Lat.to_numpy()[0],vacation_start.Lng.to_numpy()[0]
end = vacation_end.Lat.to_numpy()[0],vacation_end.Lng.to_numpy()[0]
stop1 = vacation_stop1.Lat.to_numpy()[0],vacation_stop1.Lng.to_numpy()[0]
stop2 = vacation_stop2.Lat.to_numpy()[0],vacation_stop2.Lng.to_numpy()[0]
stop3 = vacation_stop3.Lat.to_numpy()[0],vacation_stop3.Lng.to_numpy()[0]

start


(-12.0667, 134.2667)

In [9]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')
fig.add_layer(vacation_itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.

itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,562,Maningrida,AU,88.95,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge
1,176,Alice Springs,AU,78.35,clear sky,-23.7000,133.8833,Desert Palms Alice Springs
2,333,Hervey Bay,AU,84.31,light rain,-25.2986,152.8535,Shelly Bay Resort
3,122,Mareeba,AU,81.32,broken clouds,-17.0000,145.4333,Trinity Plains Tourist Park


In [13]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
itinerary_locations = itinerary_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map.

marker_layer = gmaps.marker_layer(itinerary_locations, info_box_content=itinerary_hotel_info)

fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))